In [2]:
import numpy as np
import pandas as pd
import pyodbc
import json
from collections import defaultdict

In [2]:
server = "192.168.10.35"
database = "MarketDB"
username = "mostafa"
password = "mostafa1234"
driver = "ODBC Driver 18 for SQL Server"

conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)

query = """SELECT [StockID]
      ,[Name]
      ,[FullName]
  FROM [MarketDB].[dbo].[SmallStocks]
"""

stocks = pd.read_sql_query(query, conn)

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_9064\3503057335.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stocks = pd.read_sql_query(query, conn)


In [3]:
stocks.head()

,StockID,Name,FullName
0,114312662654155,وسپهر,سرمایه گذاری مالی سپهرصادرات
1,118005828419984,چتر,صندوق س.بخشی گستره فیروزه2 ب
2,204092872752957,شصدف,صنعتی دوده فام
3,318005355896147,فسا,پتروشیمی فسا
4,383529400851043,الکتروماد,صنایع الکترونیک مادیران


In [4]:
server = "192.168.10.35"
database = "MarketHistoryDb"
username = "mostafa"
password = "mostafa1234"
driver = "ODBC Driver 18 for SQL Server"

conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)

df = pd.DataFrame([])

    
# for stock in stocks.iloc[: , 0]:
#     query = f"""SELECT [StockID]
#       ,[Date]
#       ,[Time]
#       ,[Count]
#       ,[Volume]
#       ,[TradsValue]
#       ,[LastTradePrice]
#       ,[FinalTradePrice]
#       ,[MinPriceTrade]
#       ,[MaxPriceTrade]
#   FROM [MarketHistoryDb].[dbo].[PriceHistories]

#   where StockID={stock} and [Date] >= DATEADD(MONTH, -1, GETDATE())
#   order by Time
#     """
#     df = pd.concat([df , pd.read_sql_query(query, conn)] , axis=0)

query = f"""SELECT [StockID]
      ,[Date]
      ,[Time]
      ,[Count]
      ,[Volume]
      ,[TradsValue]
      ,[LastTradePrice]
      ,[FinalTradePrice]
      ,[MinPriceTrade]
      ,[MaxPriceTrade]
  FROM [MarketHistoryDb].[dbo].[PriceHistories]

  where [Date] >= DATEADD(MONTH, -1, GETDATE())
  ORDER BY StockID, Date , Time
    """
df = pd.read_sql_query(query, conn)
    
df.to_csv("prices_data.csv", index=False, encoding="utf-8-sig")

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_9064\1103182346.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [5]:
df["Date"] = pd.to_datetime(df["Date"])

In [6]:
df.head()

,StockID,Date,Time,Count,Volume,TradsValue,LastTradePrice,FinalTradePrice,MinPriceTrade,MaxPriceTrade
0,114312662654155,2025-10-18,09:00:01,4,41000,148379000,3619,3548,3619,3619
1,114312662654155,2025-10-18,09:00:13,5,42475,153615250,3550,3548,3550,3619
2,114312662654155,2025-10-18,09:01:29,6,45475,164445250,3610,3548,3550,3619
3,114312662654155,2025-10-18,09:01:48,7,65475,235585250,3557,3548,3550,3619
4,114312662654155,2025-10-18,09:02:24,9,72337,260357070,3610,3548,3550,3619


In [7]:
df['LastTradePrice'] = pd.to_numeric(df['LastTradePrice'], errors='coerce')

df['PriceDiff'] = df.groupby('StockID')['LastTradePrice'].diff().abs()

df_positive_diff = df[df['PriceDiff'] > 0]

result_df = df_positive_diff.groupby('StockID')['PriceDiff'].min().reset_index()

result_df.rename(columns={'PriceDiff': 'MinPositiveDiff'}, inplace=True)



In [8]:
result_df

,StockID,MinPositiveDiff
0,114312662654155,1.0
1,118005828419984,1.0
2,204092872752957,10.0
3,318005355896147,1.0
4,383529400851043,1.0
...,...,...
1007,71843282162462661,1.0
1008,71856634742001725,10.0
1009,71945594172117613,1.0
1010,71957984642204570,1.0


In [9]:
result_df.to_csv("result_price_history.csv" ,index=False, encoding="utf-8-sig")

In [10]:
result_df = pd.read_csv("result_price_history.csv")

In [15]:
orders_df_org = pd.read_csv("../order_books/orders.csv")

In [23]:
orders_df = orders_df_org.copy()

In [27]:
orders_df.head()

,StockID,Date,orders
0,114312662654155,2025-10-22,"[[60124,1,1,1,3392,10000],[60124,1,2,1,3527,15..."
1,114312662654155,2025-10-21,"[[60125,2,1,1,3348,2000],[60125,2,2,1,3631,137..."
2,114312662654155,2025-10-20,"[[60122,1,1,1,3442,8000],[60122,1,2,1,3590,100..."
3,114312662654155,2025-10-19,"[[60124,1,1,1,3450,5000],[60124,1,2,1,3825,110..."
4,114312662654155,2025-10-18,"[[60125,2,1,1,3450,5000],[60125,2,2,2,3916,220..."


In [25]:
orders_df["Date"] = pd.to_datetime(orders_df["Date"])

In [26]:
today = pd.Timestamp.today()
one_month_ago = today - pd.DateOffset(months=2)

orders_df = orders_df[(orders_df["Date"] >= one_month_ago) & (orders_df["Date"] < today)]


In [28]:
def extract_max_value(row):
    arr = json.loads(row["orders"] )
    filtered = [item for item in arr if item[3] == 1] 
    
    if not filtered:   
        return None
    
    best_item = max(filtered, key=lambda x: x[5])
    
    return best_item[5]  

orders_df["MaximumOrderCount"] = orders_df.apply(extract_max_value, axis=1)


In [36]:
orders_df_idx = orders_df.groupby("StockID")["MaximumOrderCount"].idxmax()
result_order_df = orders_df.loc[orders_df_idx].reset_index(drop=True)

In [39]:
result_order_df

,StockID,Date,orders,MaximumOrderCount
0,114312662654155,2025-10-19,"[[60124,1,1,1,3450,5000],[60124,1,2,1,3825,110...",500000.0
1,118005828419984,2025-10-12,"[[60713,3,1,0,0,0],[60713,3,2,0,0,0],[60713,1,...",1000000.0
2,204092872752957,2025-10-22,"[[60124,1,1,1,11760,5000],[60124,1,2,1,12220,4...",100000.0
3,318005355896147,2025-10-22,"[[60124,1,1,0,0,0],[60124,1,2,0,0,0],[60124,5,...",800000.0
4,383529400851043,2025-10-21,"[[60125,2,1,1,3732,10000],[60125,2,2,1,4300,38...",400000.0
...,...,...,...,...
1000,71843282162462661,2025-10-07,"[[60123,1,1,1,27102,110672],[60123,1,2,1,27300...",400000000.0
1001,71856634742001725,2025-10-22,"[[60124,1,1,2,12610,2806],[60124,1,2,3,13000,4...",100000.0
1002,71945594172117613,2025-10-22,"[[60124,1,1,1,21685,500],[60124,1,2,1,22122,30...",1000000.0
1003,71957984642204570,2025-10-22,"[[60124,1,1,1,1021,100000],[60124,1,2,1,1059,2...",800000.0


In [40]:
orders_df_for_save = result_order_df[["StockID" , "Date" , "MaximumOrderCount"]]

In [41]:
orders_df_for_save

,StockID,Date,MaximumOrderCount
0,114312662654155,2025-10-19,500000.0
1,118005828419984,2025-10-12,1000000.0
2,204092872752957,2025-10-22,100000.0
3,318005355896147,2025-10-22,800000.0
4,383529400851043,2025-10-21,400000.0
...,...,...,...
1000,71843282162462661,2025-10-07,400000000.0
1001,71856634742001725,2025-10-22,100000.0
1002,71945594172117613,2025-10-22,1000000.0
1003,71957984642204570,2025-10-22,800000.0


In [42]:
orders_df_for_save.to_csv("result_max_orders.csv")

In [43]:
result_df = result_df.merge(orders_df[["StockID", "MaximumOrderCount"]], on="StockID", how="left")

In [48]:
result_df.sample(10)

,StockID,MinPositiveDiff,MaximumOrderCount
17864,70934270174405743,10.0,100000.0
16102,64659064562132185,1.0,50000000.0
8939,34718633636164421,1.0,300000000.0
12810,50185721305191887,1.0,400000.0
453,2019449432639594,1.0,100000.0
6525,26543014712914772,1.0,400000.0
3178,12901875871456398,1.0,400000.0
1555,6110133418282108,1.0,400000.0
16326,65190029034514887,1.0,20000000.0
10752,42799209630949274,10.0,762089.0


In [49]:
result_df.shape

(18251, 3)

In [50]:
orders_df.shape

(18263, 4)

In [51]:
from sqlalchemy import create_engine
import urllib.parse


database = "Boors"
server = "njweb"
username = "Boors"
password = urllib.parse.quote_plus("Boors2025@")
driver = "ODBC+Driver+18+for+SQL+Server"

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver}"
    "&Encrypt=no"
    "&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

with engine.connect() as conn:
    result_df.to_sql("priceHistory", con=conn, if_exists="replace", index=False)